# 1. Get variables

In [ ]:
keyvaultlsname = 'Ls_KeyVault_01'
adls2lsname = 'Ls_AdlsGen2_01'

# 2. Linked Services Setup: KV and ADLS Gen2

In [ ]:
from pyspark.sql import SparkSession

sc = SparkSession.builder.getOrCreate()
token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary
storage_account_url = token_library.getSecretWithLS(keyvaultlsname, "datalakeurl").replace("https://", '')

spark.conf.set("spark.storage.synapse.linkedServiceName", adls2lsname)
spark.conf.set("fs.azure.account.oauth.provider.type", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")


# 3. Create Schemas

In [ ]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS dw LOCATION 'abfss://datalake@{storage_account_url}/data'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS lnd LOCATION 'abfss://datalake@{storage_account_url}/data'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS interim LOCATION 'abfss://datalake@{storage_account_url}/data'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS malformed LOCATION 'abfss://datalake@{storage_account_url}/data'")

# 4. Create Fact Tables

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS dw.fact_parking")

spark.sql(f"CREATE TABLE dw.fact_parking(dim_date_id STRING,dim_time_id STRING, dim_parking_bay_id STRING, dim_location_id STRING, dim_st_marker_id STRING, status STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account_url}/data/dw/fact_parking/'")
 
spark.sql(f"REFRESH TABLE dw.fact_parking")

# 5. Create Dimension Tables

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS dw.dim_st_marker")
spark.sql(f"CREATE TABLE dw.dim_st_marker(dim_st_marker_id STRING, st_marker_id STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account_url}/data/dw/dim_st_marker/'")
spark.sql(f"REFRESH TABLE dw.dim_st_marker")
 

spark.sql(f"DROP TABLE IF EXISTS dw.dim_location")
spark.sql(f"CREATE TABLE dw.dim_location(dim_location_id STRING,lat FLOAT, lon FLOAT, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account_url}/data/dw/dim_location/'")
spark.sql(f"REFRESH TABLE dw.dim_location")
 

spark.sql(f"DROP TABLE IF EXISTS dw.dim_parking_bay")
spark.sql(f"CREATE TABLE dw.dim_parking_bay(dim_parking_bay_id STRING, bay_id INT,`marker_id` STRING, `meter_id` STRING, `rd_seg_dsc` STRING, `rd_seg_id` STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account_url}/data/dw/dim_parking_bay/'")
spark.sql(f"REFRESH TABLE dw.dim_parking_bay")

# 6. Create dim date and time

In [ ]:
from pyspark.sql.functions import col
spark.sql(f"DROP TABLE IF EXISTS dw.dim_date")
spark.sql(f"DROP TABLE IF EXISTS dw.dim_time")

# DimDate
dimdate = spark.read.csv(f"abfss://datalake@{storage_account_url}/data/seed/dim_date/dim_date.csv", header=True)
dimdate.write.saveAsTable("dw.dim_date")

# DimTime
dimtime = spark.read.csv(f"abfss://datalake@{storage_account_url}/data/seed/dim_time/dim_time.csv", header=True)
dimtime.write.saveAsTable("dw.dim_time")

# 7. Create interim and error tables

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS interim.parking_bay")
spark.sql(f"CREATE TABLE interim.parking_bay(bay_id INT, `last_edit` TIMESTAMP, `marker_id` STRING, `meter_id` STRING, `rd_seg_dsc` STRING, `rd_seg_id` STRING, `the_geom` STRUCT<`coordinates`: ARRAY<ARRAY<ARRAY<ARRAY<DOUBLE>>>>, `type`: STRING>, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account_url}/data/interim/interim.parking_bay/'")
spark.sql(f"REFRESH TABLE interim.parking_bay")

spark.sql(f"DROP TABLE IF EXISTS interim.sensor")
spark.sql(f"CREATE TABLE  interim.sensor(bay_id INT, `st_marker_id` STRING, `lat` FLOAT, `lon` FLOAT, `location` STRUCT<`coordinates`: ARRAY<DOUBLE>, `type`: STRING>, `status` STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account_url}/data/interim/interim.sensor/'")
spark.sql(f"REFRESH TABLE  interim.sensor")
   

spark.sql(f"DROP TABLE IF EXISTS malformed.parking_bay")
spark.sql(f"CREATE TABLE malformed.parking_bay(bay_id INT, `last_edit` TIMESTAMP,`marker_id` STRING, `meter_id` STRING, `rd_seg_dsc` STRING, `rd_seg_id` STRING, `the_geom` STRUCT<`coordinates`: ARRAY<ARRAY<ARRAY<ARRAY<DOUBLE>>>>, `type`: STRING>, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account_url}/data/malformed/malformed.parking_bay/'")
spark.sql(f"REFRESH TABLE malformed.parking_bay")

spark.sql(f"DROP TABLE IF EXISTS malformed.sensor")
spark.sql(f"CREATE TABLE malformed.sensor(bay_id INT,`st_marker_id` STRING,`lat` FLOAT,`lon` FLOAT,`location` STRUCT<`coordinates`: ARRAY<DOUBLE>, `type`: STRING>,`status` STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account_url}/data/malformed/malformed.parking_bay/'")
spark.sql(f"REFRESH TABLE malformed.sensor")